In [1]:
!python -V

Python 3.11.7


In [2]:
import pandas as pd
import pickle
from sklearn. feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [3]:
def get_dataframe(filename):

    df =pd.read_parquet(filename)
    print("The number of columns before was ",len(df.columns))
    df['duration'] = df.lpep_dropoff_datetime-df.lpep_pickup_datetime  
    df['duration'] = df.lpep_dropoff_datetime-df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda x :x.total_seconds()/60)
    print("The SD for duration is ",df.duration.std())
    old = df.shape[0]
    df=df[(df.duration >=1 ) & (df.duration <=60)]
    new=df.shape[0]
    perc = new/old *100
    print("The number of columns now is ",len(df.columns))
    print(" The number of retained records is ", perc)
    categorical=['PULocationID' ,'DOLocationID']
    df[categorical] =df[categorical].astype(str)
    print(df.info())
    return df

In [4]:
dft = get_dataframe('/home/nikki/data/green_tripdata_2023-01.parquet')


The number of columns before was  20
The SD for duration is  74.92563122537368
The number of columns now is  21
 The number of retained records is  96.6794212077231
<class 'pandas.core.frame.DataFrame'>
Index: 65946 entries, 0 to 68210
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               65946 non-null  int64         
 1   lpep_pickup_datetime   65946 non-null  datetime64[us]
 2   lpep_dropoff_datetime  65946 non-null  datetime64[us]
 3   store_and_fwd_flag     61656 non-null  object        
 4   RatecodeID             61656 non-null  float64       
 5   PULocationID           65946 non-null  object        
 6   DOLocationID           65946 non-null  object        
 7   passenger_count        61656 non-null  float64       
 8   trip_distance          65946 non-null  float64       
 9   fare_amount            65946 non-null  float64       
 10  extra             

In [5]:
dfv= get_dataframe('/home/nikki/data/green_tripdata_2023-02.parquet')

The number of columns before was  20
The SD for duration is  69.28096981389244
The number of columns now is  21
 The number of retained records is  96.55140489746795
<class 'pandas.core.frame.DataFrame'>
Index: 62574 entries, 0 to 64808
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               62574 non-null  int32         
 1   lpep_pickup_datetime   62574 non-null  datetime64[us]
 2   lpep_dropoff_datetime  62574 non-null  datetime64[us]
 3   store_and_fwd_flag     57827 non-null  object        
 4   RatecodeID             57827 non-null  float64       
 5   PULocationID           62574 non-null  object        
 6   DOLocationID           62574 non-null  object        
 7   passenger_count        57827 non-null  float64       
 8   trip_distance          62574 non-null  float64       
 9   fare_amount            62574 non-null  float64       
 10  extra            

In [6]:
numerical=['trip_distance']
categorical =['PULocationID' ,'DOLocationID']
print(dft[categorical].dtypes)
print(dfv[categorical].dtypes)

PULocationID    object
DOLocationID    object
dtype: object
PULocationID    object
DOLocationID    object
dtype: object


In [7]:

#dft['PU_DO'] =dft['PULocationID']+ '_'+dft['DOLocationID']
#dfv['PU_DO'] =dfv['PULocationID']+ '_'+dfv['DOLocationID']


dv= DictVectorizer()
target = 'duration'

train_dicts =dft[categorical + numerical].to_dict(orient='records')
X_train =dv.fit_transform(train_dicts)
y_train=dft[target].values


val_dicts =dfv[categorical + numerical].to_dict(orient='records')
X_val =dv.transform(val_dicts)

y_val=dfv[target].values

In [8]:
len(dv.feature_names_)

468

In [9]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred= lr.predict(X_train)
mean_squared_error(y_train, y_pred,squared=False)

7.060351532414825

In [10]:
y_pred= lr.predict(X_val)
mean_squared_error(y_val, y_pred,squared=False)

7.3553759711175255

In [11]:
ls =Lasso(alpha =0.01)
ls.fit(X_train,y_train)
y_pred= ls.predict(X_train)
mean_squared_error(y_train, y_pred,squared=False)

7.493687548081052

In [12]:
y_pred= ls.predict(X_val)
mean_squared_error(y_val, y_pred,squared=False)

7.740541267630343

In [13]:
with open('/home/nikki/models/linreg2.bin','wb') as f_out:
    pickle.dump((dv,lr),f_out)